# Playing the audio file

In [1]:
from IPython.display import Audio
from scipy.io import wavfile
import numpy as np

In [2]:
file_name = 'sample.wav'

In [3]:
Audio(file_name)

In [4]:
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sample rate:',framerate,'Hz')
print('Total time:',len(sounddata)/framerate,'s')

Sample rate: 48000 Hz
Total time: 3.98 s


# Generating Text from Audio

In [ ]:
!pip install transformers 

In [33]:
import soundfile as sf
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [34]:
# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
input_audio, _ = librosa.load(file_name, 
                              sr=16000)

In [19]:
input_values = processor(input_audio, return_tensors="pt", sampling_rate=16000).input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

In [35]:
transcription.lower()

'hallo my name is enty'